# Github: initial fetch

In [11]:
%load_ext rich

In [29]:
import os
import json
import shutil
from pathlib import Path

from rich import print

import requests
import requests_cache

from nya.config import settings

In [47]:
session_backend = requests_cache.SQLiteCache(db_path=settings.github_cache_path)
session = requests_cache.CachedSession(
    backend=session_backend,
    expire_after= {
        '*.github.com': 360,  # Placeholder expiration; should be overridden by Cache-Control
        '*': requests_cache.DO_NOT_CACHE,  # Don't cache anything other than GitHub requests
    },#0.0001, #settings.github_cache_expires,
    should_strip_auth=True,
    ignored_parameters=["Authorization","authorization"],
    cache_control=True,
)

# Get a repository

- [api docs](https://docs.github.com/en/rest/repos/repos#get-a-repository)

In [48]:
owner = "PyGithub"
repo = "PyGithub"

url = "https://api.github.com/repos/{owner:}/{repo:}".format(owner=owner, repo=repo)

headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {settings.github_token.get_secret_value()}",
    "X-GitHub-Api-Version": "2022-11-28"
}

res = session.get(url, headers=headers)

# res.raise_for_status()

In [49]:
res.text

'{"id":3544490,"node_id":"MDEwOlJlcG9zaXRvcnkzNTQ0NDkw","name":"PyGithub","full_name":"PyGithub/PyGithub","private":false,"owner":{"login":"PyGithub","id":11288996,"node_id":"MDEyOk9yZ2FuaXphdGlvbjExMjg4OTk2","avatar_url":"https://avatars.githubusercontent.com/u/11288996?v=4","gravatar_id":"","url":"https://api.github.com/users/PyGithub","html_url":"https://github.com/PyGithub","followers_url":"https://api.github.com/users/PyGithub/followers","following_url":"https://api.github.com/users/PyGithub/following{/other_user}","gists_url":"https://api.github.com/users/PyGithub/gists{/gist_id}","starred_url":"https://api.github.com/users/PyGithub/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/PyGithub/subscriptions","organizations_url":"https://api.github.com/users/PyGithub/orgs","repos_url":"https://api.github.com/users/PyGithub/repos","events_url":"https://api.github.com/users/PyGithub/events{/privacy}","received_events_url":"https://api.github.com/users/PyGithub/received_events","type":"Organization","site_admin":false},"html_url":"https://github.com/PyGithub/PyGithub","description":"Typed interactions with the GitHub API v3","fork":false,"url":"https://api.github.com/repos/PyGithub/PyGithub","forks_url":"https://api.github.com/repos/PyGithub/PyGithub/forks","keys_url":"https://api.github.com/repos/PyGithub/PyGithub/keys{/key_id}","collaborators_url":"https://api.github.com/repos/PyGithub/PyGithub/collaborators{/collaborator}","teams_url":"https://api.github.com/repos/PyGithub/PyGithub/teams","hooks_url":"https://api.github.com/repos/PyGithub/PyGithub/hooks","issue_events_url":"https://api.github.com/repos/PyGithub/PyGithub/issues/events{/number}","events_url":"https://api.github.com/repos/PyGithub/PyGithub/events","assignees_url":"https://api.github.com/repos/PyGithub/PyGithub/assignees{/user}","branches_url":"https://api.github.com/repos/PyGithub/PyGithub/branches{/branch}","tags_url":"https://api.github.com/repos/PyGithub/PyGithub/tags","blobs_url":"https://api.github.com/repos/PyGithub/PyGithub/git/blobs{/sha}","git_tags_url":"https://api.github.com/repos/PyGithub/PyGithub/git/tags{/sha}","git_refs_url":"https://api.github.com/repos/PyGithub/PyGithub/git/refs{/sha}","trees_url":"https://api.github.com/repos/PyGithub/PyGithub/git/trees{/sha}","statuses_url":"https://api.github.com/repos/PyGithub/PyGithub/statuses/{sha}","languages_url":"https://api.github.com/repos/PyGithub/PyGithub/languages","stargazers_url":"https://api.github.com/repos/PyGithub/PyGithub/stargazers","contributors_url":"https://api.github.com/repos/PyGithub/PyGithub/contributors","subscribers_url":"https://api.github.com/repos/PyGithub/PyGithub/subscribers","subscription_url":"https://api.github.com/repos/PyGithub/PyGithub/subscription","commits_url":"https://api.github.com/repos/PyGithub/PyGithub/commits{/sha}","git_commits_url":"https://api.github.com/repos/PyGithub/PyGithub/git/commits{/sha}","comments_url":"https://api.github.com/repos/PyGithub/PyGithub/comments{/number}","issue_comment_url":"https://api.github.com/repos/PyGithub/PyGithub/issues/comments{/number}","contents_url":"https://api.github.com/repos/PyGithub/PyGithub/contents/{+path}","compare_url":"https://api.github.com/repos/PyGithub/PyGithub/compare/{base}...{head}","merges_url":"https://api.github.com/repos/PyGithub/PyGithub/merges","archive_url":"https://api.github.com/repos/PyGithub/PyGithub/{archive_format}{/ref}","downloads_url":"https://api.github.com/repos/PyGithub/PyGithub/downloads","issues_url":"https://api.github.com/repos/PyGithub/PyGithub/issues{/number}","pulls_url":"https://api.github.com/repos/PyGithub/PyGithub/pulls{/number}","milestones_url":"https://api.github.com/repos/PyGithub/PyGithub/milestones{/number}","notifications_url":"https://api.github.com/repos/PyGithub/PyGithub/notifications{?since,all,participating}","labels_url":"https://api.github.com/repos/PyGithub/PyGithub/labels{/name}","releases_url":"https://api.github.com/repos/PyGithub/PyGit

In [62]:
print(res.request.headers)

{'Accept': 'application/vnd.github+json', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'keep-alive', 
'User-Agent': 'python-requests/2.28.1', 'X-GitHub-Api-Version': '2022-11-28'}

In [52]:
out

{
    'id': 3544490,
    'node_id': 'MDEwOlJlcG9zaXRvcnkzNTQ0NDkw',
    'name': 'PyGithub',
    'full_name': 'PyGithub/PyGithub',
    'private': False,
    'owner': {
        'login': 'PyGithub',
        'id': 11288996,
        'node_id': 'MDEyOk9yZ2FuaXphdGlvbjExMjg4OTk2',
        'avatar_url': 'https://avatars.githubusercontent.com/u/11288996?v=4',
        'gravatar_id': '',
        'url': 'https://api.github.com/users/PyGithub',
        'html_url': 'https://github.com/PyGithub',
        'followers_url': 'https://api.github.com/users/PyGithub/followers',
        'following_url': 'https://api.github.com/users/PyGithub/following{/other_user}',
        'gists_url': 'https://api.github.com/users/PyGithub/gists{/gist_id}',
        'starred_url': 'https://api.github.com/users/PyGithub/starred{/owner}{/repo}',
        'subscriptions_url': 'https://api.github.com/users/PyGithub/subscriptions',
        'organizations_url': 'https://api.github.com/users/PyGithub/orgs',
        'repos_url': 'https://api.github.com/users/PyGithub/repos',
        'events_url': 'https://api.github.com/users/PyGithub/events{/privacy}',
        'received_events_url': 'https://api.github.com/users/PyGithub/received_events',
        'type': 'Organization',
        'site_admin': False
    },
    'html_url': 'https://github.com/PyGithub/PyGithub',
    'description': 'Typed interactions with the GitHub API v3',
    'fork': False,
    'url': 'https://api.github.com/repos/PyGithub/PyGithub',
    'forks_url': 'https://api.github.com/repos/PyGithub/PyGithub/forks',
    'keys_url': 'https://api.github.com/repos/PyGithub/PyGithub/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/PyGithub/PyGithub/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/PyGithub/PyGithub/teams',
    'hooks_url': 'https://api.github.com/repos/PyGithub/PyGithub/hooks',
    'issue_events_url': 'https://api.github.com/repos/PyGithub/PyGithub/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/PyGithub/PyGithub/events',
    'assignees_url': 'https://api.github.com/repos/PyGithub/PyGithub/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/PyGithub/PyGithub/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/PyGithub/PyGithub/tags',
    'blobs_url': 'https://api.github.com/repos/PyGithub/PyGithub/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/PyGithub/PyGithub/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/PyGithub/PyGithub/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/PyGithub/PyGithub/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/PyGithub/PyGithub/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/PyGithub/PyGithub/languages',
    'stargazers_url': 'https://api.github.com/repos/PyGithub/PyGithub/stargazers',
    'contributors_url': 'https://api.github.com/repos/PyGithub/PyGithub/contributors',
    'subscribers_url': 'https://api.github.com/repos/PyGithub/PyGithub/subscribers',
    'subscription_url': 'https://api.github.com/repos/PyGithub/PyGithub/subscription',
    'commits_url': 'https://api.github.com/repos/PyGithub/PyGithub/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/PyGithub/PyGithub/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/PyGithub/PyGithub/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/PyGithub/PyGithub/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/PyGithub/PyGithub/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/PyGithub/PyGithub/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/PyGithub/PyGithub/merges',
    'archive_url': 'https://api.github.com/repos/PyGithub/PyGithub/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/PyGithub/PyGithub/downloads',
    'issues_url': 'https://api.github.com/repos/PyGithub/PyGithub

In [53]:
res

CachedResponse(
    _content=b'{"id":3544490,"node_id":"MDEwOlJlcG9zaXRvcnkzNTQ0NDkw","name":"PyGithub","full_name":"PyGithub/PyGithub","private":false,"owner":{"login":"PyGithub","id":11288996,"node_id":"MDEyOk9yZ2FuaXphdGlvbjExMjg4OTk2","avatar_url":"https://avatars.githubusercontent.com/u/11288996?v=4","gravatar_id":"","url":"https://api.github.com/users/PyGithub","html_url":"https://github.com/PyGithub","followers_url":"https://api.github.com/users/PyGithub/followers","following_url":"https://api.github.com/users/PyGithub/following{/other_user}","gists_url":"https://api.github.com/users/PyGithub/gists{/gist_id}","starred_url":"https://api.github.com/users/PyGithub/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/PyGithub/subscriptions","organizations_url":"https://api.github.com/users/PyGithub/orgs","repos_url":"https://api.github.com/users/PyGithub/repos","events_url":"https://api.github.com/users/PyGithub/events{/privacy}","received_events_url":"https://api.github.com/users/PyGithub/received_events","type":"Organization","site_admin":false},"html_url":"https://github.com/PyGithub/PyGithub","description":"Typed interactions with the GitHub API v3","fork":false,"url":"https://api.github.com/repos/PyGithub/PyGithub","forks_url":"https://api.github.com/repos/PyGithub/PyGithub/forks","keys_url":"https://api.github.com/repos/PyGithub/PyGithub/keys{/key_id}","collaborators_url":"https://api.github.com/repos/PyGithub/PyGithub/collaborators{/collaborator}","teams_url":"https://api.github.com/repos/PyGithub/PyGithub/teams","hooks_url":"https://api.github.com/repos/PyGithub/PyGithub/hooks","issue_events_url":"https://api.github.com/repos/PyGithub/PyGithub/issues/events{/number}","events_url":"https://api.github.com/repos/PyGithub/PyGithub/events","assignees_url":"https://api.github.com/repos/PyGithub/PyGithub/assignees{/user}","branches_url":"https://api.github.com/repos/PyGithub/PyGithub/branches{/branch}","tags_url":"https://api.github.com/repos/PyGithub/PyGithub/tags","blobs_url":"https://api.github.com/repos/PyGithub/PyGithub/git/blobs{/sha}","git_tags_url":"https://api.github.com/repos/PyGithub/PyGithub/git/tags{/sha}","git_refs_url":"https://api.github.com/repos/PyGithub/PyGithub/git/refs{/sha}","trees_url":"https://api.github.com/repos/PyGithub/PyGithub/git/trees{/sha}","statuses_url":"https://api.github.com/repos/PyGithub/PyGithub/statuses/{sha}","languages_url":"https://api.github.com/repos/PyGithub/PyGithub/languages","stargazers_url":"https://api.github.com/repos/PyGithub/PyGithub/stargazers","contributors_url":"https://api.github.com/repos/PyGithub/PyGithub/contributors","subscribers_url":"https://api.github.com/repos/PyGithub/PyGithub/subscribers","subscription_url":"https://api.github.com/repos/PyGithub/PyGithub/subscription","commits_url":"https://api.github.com/repos/PyGithub/PyGithub/commits{/sha}","git_commits_url":"https://api.github.com/repos/PyGithub/PyGithub/git/commits{/sha}","comments_url":"https://api.github.com/repos/PyGithub/PyGithub/comments{/number}","issue_comment_url":"https://api.github.com/repos/PyGithub/PyGithub/issues/comments{/number}","contents_url":"https://api.github.com/repos/PyGithub/PyGithub/contents/{+path}","compare_url":"https://api.github.com/repos/PyGithub/PyGithub/compare/{base}...{head}","merges_url":"https://api.github.com/repos/PyGithub/PyGithub/merges","archive_url":"https://api.github.com/repos/PyGithub/PyGithub/{archive_format}{/ref}","downloads_url":"https://api.github.com/repos/PyGithub/PyGithub/downloads","issues_url":"https://api.github.com/repos/PyGithub/PyGithub/issues{/number}","pulls_url":"https://api.github.com/repos/PyGithub/PyGithub/pulls{/number}","milestones_url":"https://api.github.com/repos/PyGithub/PyGithub/milestones{/number}","notifications_url":"https://api.github.com/repos/PyGithub/PyGithub/notifications{?since,all,participating}","labels_url":"https://api.github.com/repos/PyGithub/PyGithub/labels{/name}","releases_url":"https://api.g

In [54]:
# {k:v for k,v in res.headers.items() if k.startswith("X-Rate")}
{k:v for k,v in res.headers.items() if k.startswith("X-")}

{
    'X-OAuth-Scopes': '',
    'X-Accepted-OAuth-Scopes': 'repo',
    'X-GitHub-Media-Type': 'github.v3; format=json',
    'X-RateLimit-Limit': '5000',
    'X-RateLimit-Remaining': '4940',
    'X-RateLimit-Reset': '1672983203',
    'X-RateLimit-Used': '60',
    'X-RateLimit-Resource': 'core',
    'X-Frame-Options': 'deny',
    'X-Content-Type-Options': 'nosniff',
    'X-XSS-Protection': '0',
    'X-GitHub-Request-Id': 'E677:513C:4C34AA3:9BF6A07:63B7AC8F'
}

In [39]:
import datetime

datetime.datetime.fromtimestamp(1672983203)

datetime.datetime(2023, 1, 6, 0, 33, 23)